## Implementing Chris Doenlen's 'Bot Or Not' Python Module

Everything from `twitter_funcs.py` was cloned from Chris' [repository](https://github.com/scrapfishies/twitter-bot-detection).

I will use this to label each user as 'bot' (boolean 1/0).

In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy
from twitter_funcs import *

# Imports
import os
import numpy as np
import pandas as pd

import pickle
import tweepy

from datetime import datetime
from secrets import api_secret_key, api_key, bearer_token
import re
import time
import csv

In [3]:
data = pd.read_pickle("pickle/n2_tokenized_eff.pick")

In [4]:
data.head(3)

,id,date,time,user_id,username,hashtags,trump,biden,original,tweet,num_tokens
181142,1323379284434669568,2020-11-02,21:39:43,2820503362,artistacriseida,[],False,True,All these articles showing that Biden is in th...,article showing lead ignore govote map showing...,16
0,1323414585995526144,2020-11-02,23:59:59,1312487180258820096,annapieters17,[],False,True,@FoxNews Lady Gaga’s a nobody. Can’t figure ou...,foxnews lady nobody figure life nobody help ga...,10
4,1323414585232293888,2020-11-02,23:59:59,2335763630,kylechwatt,[],False,True,@The_Grupp “It is purely a fortuity that this ...,purely fortuity mass casualty event history ch...,8


In [5]:
def chunks(user_ids, n):
    """Yield successive n-sized chunks from user_ids (iterable)."""
    lst = list(user_ids)
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
    


I'm getting a rate limit error. according to twitter site, I can lookup 300 users per 15 minutes. Let's try that.

In [7]:
with open("pickle/bot_model.pick", "rb") as read_file:
    xgb_model = pickle.load(read_file)

In [ ]:
time.sleep(60*15)

In [ ]:
verifieds = {}
bot_probas = {}

all_users = list(data.user_id.unique())
print(f"Total number of users to scrape: {len(all_users)}")
exist = pd.read_csv("../data/user_stats.csv")
exist_users = list(exist.user_id.unique())
print(f"Number of users already scraped: {len(exist_users)}")

user_ids = []
for user in exist_users:
    if user in all_users:
        continue
    else:
        user_ids.append(user)
    
print(f"Preparing to identify bots for {len(user_ids)} users...")

user_id_chunks = list(chunks(user_ids, n=300))

# now get stats for new users

csv_file = open("../data/user_stats.csv", "a")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["user_id", "bot_proba", "verified"])
for chunk in user_id_chunks:
    print(f"Preparing chunk. Num users: {len(chunk)}")
    for user_id in chunk:
        print(f"Preparing user '{user_id}'")
        
        
        auth = tweepy.OAuthHandler(api_key, api_secret_key)
        api = tweepy.API(auth)
        
        try: # Gather features for bot/not bot model
            # Get user information from screen name
            user = api.get_user(user_id)

            # account features to return for predicton
            account_age_days = (datetime.now() - user.created_at).days
            verified = user.verified # will also use this in our data
            geo_enabled = user.geo_enabled
            default_profile = user.default_profile
            default_profile_image = user.default_profile_image
            favourites_count = user.favourites_count
            followers_count = user.followers_count
            friends_count = user.friends_count
            statuses_count = user.statuses_count
            average_tweets_per_day = np.round(statuses_count / account_age_days, 3)

            # manufactured features
            hour_created = int(user.created_at.strftime("%H"))
            network = np.round(np.log(1 + friends_count) * np.log(1 + followers_count), 3)
            tweet_to_followers = np.round(
                np.log(1 + statuses_count) * np.log(1 + followers_count), 3
            )
            follower_acq_rate = np.round(
                np.log(1 + (followers_count / account_age_days)), 3
            )
            friends_acq_rate = np.round(np.log(1 + (friends_count / account_age_days)), 3)

            # organizing list to be returned
            account_features = [
                verified, hour_created,geo_enabled,default_profile,default_profile_image,favourites_count,
                followers_count,friends_count,statuses_count,average_tweets_per_day,network,tweet_to_followers,
                follower_acq_rate,friends_acq_rate]

            if account_features == np.nan:
                proba = np.nan
                verified = np.nan
                csv_writer.writerow([user_id, proba, verified])
                continue

            else:
                user_m = np.matrix(account_features)
                proba = np.round(xgb_model.predict_proba(user_m)[:, 1][0] * 100, 2)
                verified = account_features[0]
                csv_writer.writerow([user_id, proba, verified])

        except:
            print(f'error encountered, skipping user {user_id}')
            proba = np.nan
            verified = np.nan
        
            csv_writer.writerow([user_id, proba, verified])
    print("Chunk complete. Waiting 15 minutes.")
    time.sleep(15*60+1)

csv_file.close()

In [8]:
# one-off copy
# time.sleep(120)
verifieds = {}
bot_probas = {}

user_ids = list(pd.read_pickle("pickle/grab_users.pick"))
    

user_id_chunks = list(chunks(user_ids, n=300))

# now get stats for new users

csv_file = open("../data/user_stats.csv", "a")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["user_id", "bot_proba", "verified"])
for chunk in user_id_chunks:
    print(f"Preparing chunk. Num users: {len(chunk)}")
    for user_id in chunk:
        print(f"Preparing user '{user_id}'")
        
        
        auth = tweepy.OAuthHandler(api_key, api_secret_key)
        api = tweepy.API(auth)
        
        try: # Gather features for bot/not bot model
            # Get user information from screen name
            user = api.get_user(user_id)

            # account features to return for predicton
            account_age_days = (datetime.now() - user.created_at).days
            verified = user.verified # will also use this in our data
            geo_enabled = user.geo_enabled
            default_profile = user.default_profile
            default_profile_image = user.default_profile_image
            favourites_count = user.favourites_count
            followers_count = user.followers_count
            friends_count = user.friends_count
            statuses_count = user.statuses_count
            average_tweets_per_day = np.round(statuses_count / account_age_days, 3)

            # manufactured features
            hour_created = int(user.created_at.strftime("%H"))
            network = np.round(np.log(1 + friends_count) * np.log(1 + followers_count), 3)
            tweet_to_followers = np.round(
                np.log(1 + statuses_count) * np.log(1 + followers_count), 3
            )
            follower_acq_rate = np.round(
                np.log(1 + (followers_count / account_age_days)), 3
            )
            friends_acq_rate = np.round(np.log(1 + (friends_count / account_age_days)), 3)

            # organizing list to be returned
            account_features = [
                verified, hour_created,geo_enabled,default_profile,default_profile_image,favourites_count,
                followers_count,friends_count,statuses_count,average_tweets_per_day,network,tweet_to_followers,
                follower_acq_rate,friends_acq_rate]

            if account_features == np.nan:
                proba = np.nan
                verified = np.nan
                csv_writer.writerow([user_id, proba, verified])
                continue

            else:
                user_m = np.matrix(account_features)
                proba = np.round(xgb_model.predict_proba(user_m)[:, 1][0] * 100, 2)
                verified = account_features[0]
                csv_writer.writerow([user_id, proba, verified])

        except:
            print(f'error encountered, skipping user {user_id}')
            proba = np.nan
            verified = np.nan
        
            csv_writer.writerow([user_id, proba, verified])
    print("Chunk complete. Waiting 15 minutes.")
    time.sleep(15*60+1)

csv_file.close()

Preparing chunk. Num users: 300
Preparing user '1143289465387081732'
Preparing user '782603330'
Preparing user '777679662310907904'
Preparing user '1408230732'
Preparing user '20406676'
Preparing user '1304172941391863817'
Preparing user '51827732'
Preparing user '3937940121'
Preparing user '8500962'
Preparing user '570097148'
Preparing user '104246758'
Preparing user '1043423847708590081'
Preparing user '31571443'
Preparing user '547892272'
Preparing user '160364793'
Preparing user '1251323699451564035'
Preparing user '20824974'
Preparing user '51183401'
Preparing user '1649886427'
Preparing user '15145034'
Preparing user '112557979'
Preparing user '3304837284'
Preparing user '832841680135081984'
Preparing user '1241019363525677057'
Preparing user '1255552876073680903'
Preparing user '1287515682641412096'
Preparing user '817761228688916480'
Preparing user '1239930170800107521'
Preparing user '283350568'
Preparing user '850046670'
Preparing user '728930499029172224'
Preparing user '444

Preparing user '1190079278857281539'
Preparing user '207656440'
Preparing user '912135002782969857'
Preparing user '212795909'
Preparing user '2362868665'
Preparing user '72592074'
Preparing user '585764087'
Preparing user '1072212454686515209'
Preparing user '1102655017042038785'
Preparing user '91808599'
Preparing user '1176325494259798017'
Preparing user '1289930821357445132'
Preparing user '34283650'
Preparing user '67035595'
Preparing user '1375297094'
Preparing user '1230089817418895360'
Preparing user '993215081256181760'
Preparing user '26965771'
Preparing user '1220666675072204801'
Preparing user '41964897'
Preparing user '219174241'
Preparing user '1016066208'
Preparing user '529718790'
Preparing user '1041056512582533121'
Preparing user '124275780'
Preparing user '1180983441044770817'
Preparing user '3167620869'
Preparing user '1234187035'
Preparing user '1309975396943880192'
Preparing user '17773224'
Preparing user '94823672'
Preparing user '26928006'
Preparing user '802581

Preparing user '1146079750433071104'
Preparing user '17437917'
Preparing user '757008901413961728'
Preparing user '31285825'
Preparing user '3396389877'
Preparing user '1204945320079175681'
Preparing user '1291911212163358720'
Preparing user '465897557'
Preparing user '825310374316634112'
Preparing user '2966660663'
Preparing user '565151401'
Preparing user '1578369842'
Preparing user '3343545432'
Preparing user '2360295679'
Preparing user '706840182788636672'
Preparing user '4754857394'
Preparing user '21549689'
Preparing user '1132095449358323715'
Preparing user '18286742'
Preparing user '1188920021625401344'
Preparing user '3834650237'
Preparing user '330361451'
Preparing user '923200791904702464'
Preparing user '1229952949691940864'
Preparing user '2335805793'
Preparing user '176514190'
Preparing user '3177941678'
Preparing user '897872740593651712'
Preparing user '37426831'
Preparing user '23645254'
Preparing user '300849011'
Preparing user '1215484675701035008'
Preparing user '73

Preparing user '767372100327903236'
Preparing user '735403478'
Preparing user '1297543003935068161'
Preparing user '1122303946385547264'
Preparing user '1307520297235439618'
Preparing user '281443650'
Preparing user '971925509285335042'
Preparing user '1363755902'
Preparing user '1131264983432597504'
Preparing user '1100277946055516160'
Preparing user '836004766262034432'
Preparing user '1214625199'
Preparing user '1051877297605799937'
Preparing user '370938830'
Preparing user '3018098473'
Preparing user '108987695'
Preparing user '1258523139606147072'
Preparing user '1127523210277396482'
Preparing user '74820061'
Preparing user '427901036'
Preparing user '1450670106'
Preparing user '172469352'
Preparing user '269076835'
Preparing user '224274349'
Preparing user '4458421461'
Preparing user '250944832'
Preparing user '49961271'
Preparing user '1276017627358347264'
Preparing user '78865371'
Preparing user '1254174805072457734'
Preparing user '898651031508324353'
Preparing user '18449140'

Preparing user '1266806126290534405'
Preparing user '50015359'
Preparing user '1078674438105821184'
Preparing user '1087298936166076417'
Preparing user '1150911242426552322'
Preparing user '273135082'
Preparing user '363307821'
Preparing user '3120948660'
Preparing user '36046951'
Preparing user '796933268499267584'
Preparing user '1063105875177586689'
Preparing user '227552105'
Preparing user '2722253653'
Preparing user '1208848039437205504'
Preparing user '1589924701'
Preparing user '6634632'
Preparing user '1153003898887692288'
Preparing user '2901851217'
Preparing user '17723942'
Preparing user '1269067340802666504'
Preparing user '3702282797'
Preparing user '765355137263173632'
Preparing user '516345781'
Preparing user '2591789040'
Preparing user '795267601957912576'
Preparing user '1911873032'
Preparing user '1275524614723002368'
Preparing user '44806272'
Preparing user '989035910460334081'
Preparing user '319931473'
Preparing user '1242501675299921921'
Preparing user '8244600887

...

Preparing user '607518496'
Preparing user '25434845'
Preparing user '440296447'
Preparing user '2997737048'
Preparing user '1351864994'
Preparing user '1303035487985176576'
Preparing user '1262231186044334080'
Preparing user '83739756'
Preparing user '1051374954409680897'
Preparing user '2922735484'
Preparing user '1190760414809919493'
Preparing user '1133301875191816192'
Preparing user '1133524243264802817'
Preparing user '1244076946293694464'
Preparing user '23229642'
Preparing user '845780194656968704'
Preparing user '981878997544939521'
Preparing user '1268746414982680578'
Preparing user '773658740142772224'
Preparing user '525972378'
Preparing user '1067003156838662144'
Preparing user '2377771243'
Preparing user '1085365067271598082'
Preparing user '50168095'
Preparing user '207673131'
Preparing user '1171604649192935425'
Preparing user '1065489848578887680'
Chunk complete. Waiting 15 minutes.
Preparing chunk. Num users: 300
Preparing user '1215834974919766016'
Preparing user '873

Preparing user '33564624'
Preparing user '1303770240510291971'
Preparing user '1310447526'
Preparing user '1260701576274083841'
Preparing user '1307103972373786627'
Preparing user '223013335'
Preparing user '3291665833'
Preparing user '2370330441'
Preparing user '815660782708989953'
Preparing user '4823335694'
Preparing user '22610257'
Preparing user '20016443'
Preparing user '1281065769648959490'
Preparing user '701360958'
Preparing user '144768842'
Preparing user '967772587366469633'
Preparing user '130310178'
Preparing user '87597317'
Preparing user '18572659'
Preparing user '37587319'
Preparing user '1097639396684296192'
Preparing user '1214330651362516992'
Preparing user '3018570797'
Preparing user '112332715'
Preparing user '16640775'
Preparing user '437511454'
Preparing user '14128579'
Preparing user '107120201'
Preparing user '552364248'
Preparing user '1554649760'
Preparing user '1250029281813692416'
Preparing user '59042110'
Preparing user '614609920'
Preparing user '82645393

Preparing user '976776900'
Preparing user '1280808607525867521'
Preparing user '1275441017026752514'
Preparing user '3315965327'
Preparing user '1121946294891819009'
Preparing user '539699081'
Preparing user '1293093750802006017'
Preparing user '1083467756'
Preparing user '863565514190143488'
Preparing user '779857710250471424'
Preparing user '69756540'
Preparing user '1051396218'
Preparing user '407092131'
Preparing user '507713475'
Preparing user '3026474129'
Preparing user '320919057'
Preparing user '231649709'
Preparing user '1283936105268514816'
Preparing user '629822787'
Preparing user '1332871628'
Preparing user '29823078'
Preparing user '16185624'
Preparing user '16887633'
Preparing user '824228998666452994'
Preparing user '1267981078557388800'
Preparing user '1184087546617303042'
Preparing user '2921076014'
Preparing user '47135740'
Preparing user '43827403'
Preparing user '823166629487976451'
Preparing user '14820511'
Preparing user '720195457'
Preparing user '121727836928238

Preparing user '77019112'
Preparing user '4865177659'
Preparing user '879147821915615233'
Preparing user '1088936911161491457'
Preparing user '948950056094371840'
Preparing user '593352059'
Preparing user '146430911'
Preparing user '1306678615711514625'
Preparing user '1144687015621025792'
Preparing user '1286309439256002560'
Preparing user '897599946639233024'
Preparing user '347422217'
Preparing user '83374478'
Preparing user '1299476377419341824'
Preparing user '1280162435811414018'
Preparing user '2523546552'
Preparing user '1273025257138421760'
Preparing user '624247876'
Preparing user '756128387937120257'
Preparing user '525000769'
Preparing user '253044272'
Preparing user '18641880'
Preparing user '2157301369'
Preparing user '1280677517117054976'
Preparing user '3360901048'
Preparing user '960971835755114496'
Preparing user '936307737956249600'
Preparing user '579577338'
Preparing user '1301320168908271620'
Preparing user '22288330'
Preparing user '1303775767189479424'
Preparing

KeyboardInterrupt: 

In [ ]:
bot_probas

In [ ]:
verifieds

In [ ]:
data['verified'] = data['user_id'].map(verifieds)
data['bot_proba'] = data['user_id'].map(bot_probas)


In [ ]:
bot_p